# PlotBot



### Imports

In [1]:
# Core
from datetime import date, timedelta, datetime, timezone
from operator import itemgetter
import json
import re

# Logging
import sys
import logging
logging.basicConfig(
    format='%(asctime)s | %(levelname)s : %(message)s',
    stream=sys.stdout) # Needed in Jupyter. Comment out if only need to see in log file.

# Scheduler
import schedule

# Numpy, Pandas and PyPlot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
sns.set_style(
    'darkgrid', 
    {
        'axes.facecolor': '0.9',
        'axes.titlesize': 'x-large',
        'figure.titlesize': 'x-large',
    }
)

# API
import tweepy
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Own
from twitter_config import *

### Constants and Globals

In [2]:
SCAN_MINUTES = 1
PAGE_SIZE = 20
PAGES = 2
TODAY = date.today().strftime('%m/%d/%y')

_AUTH = tweepy.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
_AUTH.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_TOKEN_SECRET)

TWITTER_API = tweepy.API(_AUTH, parser=tweepy.parsers.JSONParser())
SENTIMENT_ANALYZER = SentimentIntensityAnalyzer()

TWITTER_USER='feng443'

DEBUG = True
LOG_FILE = 'PlotBot.log'

### Define PlotBot Class

In [ ]:
class PlotBot(object):
    
    _analyzed = []
    _to_analyze = []
    _df = pd.DataFrame(index=pd.Index(
        list(range(-PAGE_SIZE * PAGES)),
        name='Tweets Ago'))
    
    @property
    def data(self):
        return self._df

    @property
    def logger(self):
        logger = logging.getLogger('PlotBot')
        if self._debug:
            logger.setLevel(logging.DEBUG)
        if self._log_file:
            logger.addHandler(logging.FileHandler(self._log_file))
        return logger
    
    def __init__(self, user=TWITTER_USER, debug=False, log_file=None,
                last_only=True):
        self._user = user
        self._re = re.compile('@PlotBot Analyze: @(\w{1,15})')
        self._debug = debug
        self._log_file = log_file
        self._last_only = last_only
        
    def _scan_tweets(self, user=TWITTER_USER, debug=True):
        self.logger.info('Scan tweets ...')
        self._tweets = []
        for tweet in TWITTER_API.user_timeline(user, count=PAGE_SIZE):
            targets = self._re.findall(tweet['text'])
            if targets:
                target = targets[0]
                if target not in self._analyzed:
                    if self._last_only and self._is_old_tweet(tweet):
                        self.logger.debug('is_old_tweet: {}'.format(self._is_old_tweet(tweet)))
                        self.logger.info('Skip older tweets.')
                        break
                    self._to_analyze.append(target)    
                    created_at = tweet['created_at']
                else:
                    self.logger.info(f'{target} already analyzed. Skip.')
    
    def _is_old_tweet(self, tweet):
        tweet_datetime = datetime.strptime(
            tweet['created_at'], "%a %b %d %H:%M:%S %z %Y")
        self.logger.debug(f'tweet time: {tweet_datetime}')
        now = datetime.now(timezone.utc) 
        self.logger.debug(f'now: {now}')
        return (now - tweet_datetime) > timedelta(minutes=SCAN_MINUTES * 2) # Times 2 to allow some processing time buffer
        
    def _analyze_all(self):
        if not self._to_analyze:
            self.logger.info('Nothing new.')
            return

        self.logger.info('Analyze all ...')
        while self._to_analyze:
            target = self._to_analyze.pop()
            self._analyzed.append(target)
            self._analyze(target)
            self._plot(target)
            self._tweet_out(target)
    
    def _analyze(self, target):
        self.logger.info(f'Analyze {target}')
        tweets_ago = 0
        for page in range(PAGES):
            for tweet in TWITTER_API.user_timeline(target, count=PAGE_SIZE, page=page):
                self._df.at[tweets_ago, 
                        f'@{target}'] = SENTIMENT_ANALYZER.polarity_scores(tweet['text'])['compound']
                tweets_ago -= 1
    
    def _plot(self, target):
        fig, ax = plt.subplots(figsize=(8, 6))
        self.data.plot.line(
            y=f'@{target}',
            marker='o',
            ax=ax,
            alpha=0.8,
        )
        plt.legend(
            title='Tweets',
            bbox_to_anchor=(1.25, 1)
        )
        plt.title(f'Sentiments Analysis of Tweet ({TODAY})',
            fontsize=16)
        plt.ylabel('Tweet Polarity')
        plt.gca().invert_xaxis()
        
        plt.savefig('plot_bot.png', bbox_inches='tight')
        plt.show()
        
    def _tweet_out(self, target):
        self.logger.info(f'Tweet out  ...{target}')
        TWITTER_API.update_with_media('plot_bot.png',
                                      f'Sentiment Analsyis of Tweets ({TODAY})')
        
    def listen(self):
        self._scan_tweets()
        self._analyze_all()

### Start the Daemon

In [ ]:
schedule.clear()
bot = PlotBot(debug=DEBUG, log_file=LOG_FILE)
schedule.every(SCAN_MINUTES).minutes.do(bot.listen)
    
while True:
    schedule.run_pending()

2018-03-18 20:23:14,004 | INFO : Scan tweets ...
2018-03-18 20:23:14,325 | DEBUG : tweet time: 2018-03-18 23:51:20+00:00
2018-03-18 20:23:14,326 | DEBUG : now: 2018-03-19 00:23:14.326177+00:00
2018-03-18 20:23:14,328 | DEBUG : tweet time: 2018-03-18 23:51:20+00:00
2018-03-18 20:23:14,330 | DEBUG : now: 2018-03-19 00:23:14.330173+00:00
2018-03-18 20:23:14,332 | DEBUG : is_old_tweet: True
2018-03-18 20:23:14,334 | INFO : Skip older tweets.
2018-03-18 20:23:14,336 | INFO : Nothing new.
2018-03-18 20:23:24,338 | INFO : Scan tweets ...
2018-03-18 20:23:24,714 | DEBUG : tweet time: 2018-03-18 23:51:20+00:00
2018-03-18 20:23:24,717 | DEBUG : now: 2018-03-19 00:23:24.716653+00:00
2018-03-18 20:23:24,719 | DEBUG : tweet time: 2018-03-18 23:51:20+00:00
2018-03-18 20:23:24,722 | DEBUG : now: 2018-03-19 00:23:24.721156+00:00
2018-03-18 20:23:24,723 | DEBUG : is_old_tweet: True
2018-03-18 20:23:24,726 | INFO : Skip older tweets.
2018-03-18 20:23:24,729 | INFO : Nothing new.
2018-03-18 20:23:34,734 

2018-03-18 20:26:01,627 | INFO : Skip older tweets.
2018-03-18 20:26:01,631 | INFO : Nothing new.
2018-03-18 20:26:11,636 | INFO : Scan tweets ...
2018-03-18 20:26:11,983 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:26:11,989 | DEBUG : now: 2018-03-19 00:26:11.988580+00:00
2018-03-18 20:26:11,995 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:26:12,002 | DEBUG : now: 2018-03-19 00:26:12.001583+00:00
2018-03-18 20:26:12,008 | DEBUG : is_old_tweet: True
2018-03-18 20:26:12,015 | INFO : Skip older tweets.
2018-03-18 20:26:12,022 | INFO : Nothing new.
2018-03-18 20:26:22,028 | INFO : Scan tweets ...
2018-03-18 20:26:22,386 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:26:22,392 | DEBUG : now: 2018-03-19 00:26:22.392495+00:00
2018-03-18 20:26:22,400 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:26:22,405 | DEBUG : now: 2018-03-19 00:26:22.405491+00:00
2018-03-18 20:26:22,412 | DEBUG : is_old_tweet: True
2018-03-18 20:26:22,419 

2018-03-18 20:28:59,283 | DEBUG : is_old_tweet: True
2018-03-18 20:28:59,293 | INFO : Skip older tweets.
2018-03-18 20:28:59,302 | INFO : Nothing new.
2018-03-18 20:29:09,312 | INFO : Scan tweets ...
2018-03-18 20:29:09,655 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:29:09,664 | DEBUG : now: 2018-03-19 00:29:09.664010+00:00
2018-03-18 20:29:09,675 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:29:09,683 | DEBUG : now: 2018-03-19 00:29:09.683003+00:00
2018-03-18 20:29:09,691 | DEBUG : is_old_tweet: True
2018-03-18 20:29:09,699 | INFO : Skip older tweets.
2018-03-18 20:29:09,707 | INFO : Nothing new.
2018-03-18 20:29:19,715 | INFO : Scan tweets ...
2018-03-18 20:29:20,089 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:29:20,096 | DEBUG : now: 2018-03-19 00:29:20.096343+00:00
2018-03-18 20:29:20,105 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:29:20,115 | DEBUG : now: 2018-03-19 00:29:20.115344+00:00
2018-03-18 20:29:20,124 

2018-03-18 20:31:57,839 | DEBUG : now: 2018-03-19 00:31:57.838970+00:00
2018-03-18 20:31:57,849 | DEBUG : is_old_tweet: True
2018-03-18 20:31:57,859 | INFO : Skip older tweets.
2018-03-18 20:31:57,871 | INFO : Nothing new.
2018-03-18 20:32:07,881 | INFO : Scan tweets ...
2018-03-18 20:32:08,282 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:32:08,290 | DEBUG : now: 2018-03-19 00:32:08.289665+00:00
2018-03-18 20:32:08,302 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:32:08,311 | DEBUG : now: 2018-03-19 00:32:08.311133+00:00
2018-03-18 20:32:08,322 | DEBUG : is_old_tweet: True
2018-03-18 20:32:08,334 | INFO : Skip older tweets.
2018-03-18 20:32:08,355 | INFO : Nothing new.
2018-03-18 20:32:18,369 | INFO : Scan tweets ...
2018-03-18 20:32:18,829 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:32:18,837 | DEBUG : now: 2018-03-19 00:32:18.836703+00:00
2018-03-18 20:32:18,848 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:32:18,858 

2018-03-18 20:34:56,862 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:34:56,878 | DEBUG : now: 2018-03-19 00:34:56.877893+00:00
2018-03-18 20:34:56,895 | DEBUG : is_old_tweet: True
2018-03-18 20:34:56,912 | INFO : Skip older tweets.
2018-03-18 20:34:56,927 | INFO : Nothing new.
2018-03-18 20:35:06,943 | INFO : Scan tweets ...
2018-03-18 20:35:07,360 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:35:07,371 | DEBUG : now: 2018-03-19 00:35:07.370739+00:00
2018-03-18 20:35:07,385 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:35:07,400 | DEBUG : now: 2018-03-19 00:35:07.399739+00:00
2018-03-18 20:35:07,413 | DEBUG : is_old_tweet: True
2018-03-18 20:35:07,427 | INFO : Skip older tweets.
2018-03-18 20:35:07,441 | INFO : Nothing new.
2018-03-18 20:35:17,456 | INFO : Scan tweets ...
2018-03-18 20:35:17,869 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:35:17,881 | DEBUG : now: 2018-03-19 00:35:17.880415+00:00
2018-03-18 20:35:17,894 

2018-03-18 20:37:55,888 | DEBUG : now: 2018-03-19 00:37:55.887731+00:00
2018-03-18 20:37:55,915 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:37:55,932 | DEBUG : now: 2018-03-19 00:37:55.930730+00:00
2018-03-18 20:37:55,948 | DEBUG : is_old_tweet: True
2018-03-18 20:37:55,969 | INFO : Skip older tweets.
2018-03-18 20:37:55,985 | INFO : Nothing new.
2018-03-18 20:38:06,002 | INFO : Scan tweets ...
2018-03-18 20:38:06,395 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:38:06,408 | DEBUG : now: 2018-03-19 00:38:06.408010+00:00
2018-03-18 20:38:06,425 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:38:06,439 | DEBUG : now: 2018-03-19 00:38:06.438550+00:00
2018-03-18 20:38:06,455 | DEBUG : is_old_tweet: True
2018-03-18 20:38:06,471 | INFO : Skip older tweets.
2018-03-18 20:38:06,485 | INFO : Nothing new.
2018-03-18 20:38:16,501 | INFO : Scan tweets ...
2018-03-18 20:38:16,906 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:38:16,920 

2018-03-18 20:40:55,407 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:40:55,431 | DEBUG : now: 2018-03-19 00:40:55.430035+00:00
2018-03-18 20:40:55,450 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:40:55,469 | DEBUG : now: 2018-03-19 00:40:55.468029+00:00
2018-03-18 20:40:55,489 | DEBUG : is_old_tweet: True
2018-03-18 20:40:55,508 | INFO : Skip older tweets.
2018-03-18 20:40:55,523 | INFO : Nothing new.
2018-03-18 20:41:05,539 | INFO : Scan tweets ...
2018-03-18 20:41:05,939 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:41:05,958 | DEBUG : now: 2018-03-19 00:41:05.958067+00:00
2018-03-18 20:41:05,986 | DEBUG : tweet time: 2018-03-18 23:41:26+00:00
2018-03-18 20:41:06,004 | DEBUG : now: 2018-03-19 00:41:06.004068+00:00
2018-03-18 20:41:06,030 | DEBUG : is_old_tweet: True
2018-03-18 20:41:06,053 | INFO : Skip older tweets.
2018-03-18 20:41:06,078 | INFO : Nothing new.
2018-03-18 20:41:16,101 | INFO : Scan tweets ...
2018-03-18 20:41:16,452 

2018-03-18 20:43:45,232 | INFO : Nothing new.
